In [35]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import random

In [36]:
class RemoveCorrelationTransformer2(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7):
        self.correlation_threshold = correlation_threshold


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson', min_periods=1)
        df_not_correlated = ~(df_corr.mask(
            np.tril(np.ones([len(df_corr)] * 2, dtype=bool))).abs() > self.correlation_threshold).any()
        self.un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df[self.un_corr_idx]
        return df.values

In [37]:

class RemoveCorrelationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7, pca_components_ratio=3):
        self.correlation_threshold = correlation_threshold
        self.pca_components_ratio = pca_components_ratio


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson')
        df_corr = df_corr - np.eye(df.shape[1])
        outliares_corr = df_corr[np.abs(df_corr) > self.correlation_threshold]
        self.outliares_corr = outliares_corr.dropna(axis=1, how='all')

        correlated_df = df[self.outliares_corr.columns]

        n_components = len(self.outliares_corr.columns) // self.pca_components_ratio
        pca = PCA(n_components=n_components)

        correlated_df = pca.fit_transform(correlated_df)
        self.correlated_df = pd.DataFrame(correlated_df, columns=["pca_{}".format(i) for i in range(n_components)])

        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        print(self.outliares_corr.columns)
        df = df.drop((self.outliares_corr.columns), axis=1)
        df = df.join(self.correlated_df)
        return df

In [38]:

class RemoveMissingFeaturesTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, Y=None):
        self.is_missing = X.isnull().values.any(axis=0)
        return self

    def transform(self, X, Y=None):
        copy_x = pd.DataFrame(X)
        self.is_missing += copy_x.isnull().values.any(axis=0)

        copy_x = copy_x.iloc[:, ~self.is_missing]

        return copy_x.values


In [39]:
def refactor_labels(df):
    return df.replace({'low': 0 ,'high': 1, 'clinical': 1 })


def get_data(file_name, LSAS_threshold=None):
    group_column = 'group'
    sub_num_col = 'Subject_Number'
    lsas_col = 'LSAS'
    df = pd.read_excel(file_name, sheet_name='Sheet1')
    if LSAS_threshold is None:
        X = df.drop([group_column, sub_num_col, lsas_col], 1)
        Y = refactor_labels(df[group_column])
        return X, Y
    else:
        
        X = df.drop([group_column], 1)
        Y = pd.Series(np.where(X[lsas_col] > LSAS_threshold, 1, 0))
        X = X.drop([sub_num_col, lsas_col], 1)
        return X, Y


In [40]:
file_name = "training_set_100.xlsx"
X_train, y_train = get_data(file_name, LSAS_threshold = 50)


In [41]:
for i in range(10):
    
    amount_features = random.randint(8, 16)

    #which features to take

    columns_shuffled = list(X_train.columns)
    random.shuffle(columns_shuffled)
    print(columns_shuffled[:amount_features])

    
    X_train = X_train[columns_shuffled[:amount_features]]
    
    X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, stratify=y_train)
    
    
    pipe =  Pipeline([
    ('correlation_threshold', RemoveCorrelationTransformer2()), 
    ('classifier', XGBClassifier())])
    
    params_grid = [
    {
    
        'classifier__max_depth': [4, 2],
        'classifier__learning_rate': [0.1, 0.05, 0.15],
        'classifier__n_estimators': [75, 200, 100]}] 
    
    
    
    cv = StratifiedKFold(10)
    gs = GridSearchCV(pipe, params_grid, cv=cv, scoring='accuracy')
    gs.fit(X_train_2, y_train_2);


    print("________________\nbest params\n",gs.best_params_)
    print("\nbest score\n",gs.best_score_)

    results = []
    k = 5
    for i in range(k):
        loo = LeaveOneOut()
        score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=loo)
        results.append(score.mean())
    print("\navg acc in train2 set\n",sum(results)/k)

    model = gs.best_estimator_.fit(X_train_2, y_train_2)
    y_pred = model.predict(X_test_2)
    print ("\nacc on holdout\n", accuracy_score(y_pred, y_test_2))
    
    results = []
    k = 5
    for i in range(k):
        loo = LeaveOneOut()
        score = cross_val_score(gs.best_estimator_, X_train, y_train, cv=loo)
        results.append(score.mean())
    print("\navg acc in train1 set\n",sum(results)/k)
    

['average_pupil_size_White_Space', 'STD_pupil_size_All', 'average_fixation_length_Neutral', 'average_fixation_length_White_Space', 'STD_fixation_length_All', 'average_pupil_size_Neutral', 'mean_different_AOI_per_trial', 'average_pupil_size_All', 'average_fixation_length_Disgusted', 'var_ratio_D_DN', 'avg_of_sum_fixation_length_Neutral', 'STD_fixation_length_White_Space']


C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


________________
best params
 {'classifier__learning_rate': 0.05, 'classifier__max_depth': 4, 'classifier__n_estimators': 75}

best score
 0.6333333333333333

avg acc in train2 set
 0.6666666666666666

acc on holdout
 0.8

avg acc in train1 set
 0.68
['STD_pupil_size_All', 'var_ratio_D_DN', 'average_fixation_length_Disgusted', 'STD_fixation_length_White_Space', 'average_pupil_size_White_Space', 'STD_fixation_length_All', 'avg_of_sum_fixation_length_Neutral', 'mean_different_AOI_per_trial', 'average_fixation_length_Neutral', 'average_pupil_size_All', 'average_pupil_size_Neutral', 'average_fixation_length_White_Space']


C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


________________
best params
 {'classifier__learning_rate': 0.1, 'classifier__max_depth': 4, 'classifier__n_estimators': 75}

best score
 0.6222222222222222

avg acc in train2 set
 0.6

acc on holdout
 0.7

avg acc in train1 set
 0.63
['STD_fixation_length_White_Space', 'var_ratio_D_DN', 'average_fixation_length_White_Space', 'average_pupil_size_Neutral', 'STD_fixation_length_All', 'average_pupil_size_White_Space', 'avg_of_sum_fixation_length_Neutral', 'average_fixation_length_Disgusted', 'average_pupil_size_All']


...

________________
best params
 {'classifier__learning_rate': 0.15, 'classifier__max_depth': 4, 'classifier__n_estimators': 200}

best score
 0.6444444444444445

avg acc in train2 set
 0.6222222222222222

acc on holdout
 0.6

avg acc in train1 set
 0.6
['STD_fixation_length_White_Space', 'avg_of_sum_fixation_length_Neutral', 'var_ratio_D_DN', 'average_fixation_length_White_Space', 'STD_fixation_length_All', 'average_pupil_size_All', 'average_fixation_length_Disgusted', 'average_pupil_size_Neutral']


C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


________________
best params
 {'classifier__learning_rate': 0.15, 'classifier__max_depth': 2, 'classifier__n_estimators': 200}

best score
 0.6888888888888889

avg acc in train2 set
 0.6222222222222222

acc on holdout
 0.7

avg acc in train1 set
 0.59
['average_pupil_size_All', 'STD_fixation_length_All', 'average_fixation_length_White_Space', 'average_pupil_size_Neutral', 'avg_of_sum_fixation_length_Neutral', 'STD_fixation_length_White_Space', 'var_ratio_D_DN', 'average_fixation_length_Disgusted']


C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


________________
best params
 {'classifier__learning_rate': 0.05, 'classifier__max_depth': 4, 'classifier__n_estimators': 75}

best score
 0.5666666666666667

avg acc in train2 set
 0.5444444444444444

acc on holdout
 0.6

avg acc in train1 set
 0.56


In [42]:
np.arange(0.2,1, 0.3)

array([0.2, 0.5, 0.8])

In [ ]:
 np.arange(0.2,0.9, 0.3)